# Project Stage 3: Entity Matching (EM)

Problem Description:
We have extracted two tables A, B. Both tables have the same schema. Now we need to perform entity matching and will do so with Magellan, an EM tool to match the two tables. 

- Goal: find entities across two tables (A, B) that match. 


In [1]:
import py_entitymatching as em
import pandas as pd
import os, sys
import numpy as np

# Read CSV

- Read csv file from disk as a table and set its metadata

In [2]:
metacriticData = pd.read_csv('data/metacritic.csv')
wikiData = pd.read_csv('data/wikiData.csv')

In [3]:
# add ID column to each dataset
metacriticID = ["a" + str(num) for num in np.arange(1, len(metacriticData.index)+1)]
wikiID = ["b" + str(num) for num in np.arange(1, len(wikiData.index)+1)]

col_idx = 0
metacriticData.insert(loc = col_idx, column = 'ID', value = metacriticID)
wikiData.insert(loc = col_idx, column = 'ID', value = wikiID)

In [4]:
metacriticData.head()

,ID,Album,Artist,Genre,Label,Producer,Release Date,Meta Score
0,a1,Wrong Creatures,Black Rebel Motorcycle Club,['Pop/Rock'],['Vagrant Records'],NaN,Jan 12 2018,69
1,a2,No Cross No Crown,Corrosion of Conformity,['Pop/Rock'],['Nuclear Blast'],NaN,Jan 12 2018,77
2,a3,Encore,Anderson East,['Singer-Songwriter'],['Low Country Sound'],NaN,Jan 12 2018,74
3,a4,A Day With The Homies [EP],Panda Bear,['Pop/Rock'],['Domino'],NaN,Jan 12 2018,74
4,a5,Four Stones,Dean McPhee,['Alternative'],['Hood Faire'],NaN,Jan 12 2018,84


In [5]:
wikiData.head()

,ID,Album,Artist,Genre,Label,Producer,Release Date,Meta Score
0,b1,Be Calm,Air Dubai,"['Hip hop', ' pop']",['Hopeless'],"['Dwight A. Baker', ' Colin Munroe']",Jul 1 2014,NaN
1,b2,From Parts Unknown,Every Time I Die,"['Metalcore', ' hardcore punk', ' mathcore', ' sludge metal']",['Epitaph'],['Kurt Ballou'],Jul 1 2014,NaN
2,b3,"I'm Almost Happy Here, But I Never Feel At Home",Hotel Books,"['Spoken word', ' indie rock', ' emo']",['inVogue'],"['Jay Maas', ' Hiram Hernandez']",Jul 1 2014,NaN
3,b4,Paula,Robin Thicke,['R&B'],"['Star Trak', ' Interscope']","['Robin Thicke', ' Pro Jay']",Jul 1 2014,NaN
4,b5,Isolate and Medicate,Seether,"['Post-grunge', ' hard rock', ' alternative metal']","['The Bicycle Music Company', ' Concord Bicycle', ' Spinefarm']","[""Brendan O'Brien""]",Jul 1 2014,NaN


In [6]:
# set metadata
em.set_key(wikiData, 'ID')
em.set_key(metacriticData, 'ID')

wikiData["Album"] = wikiData["Album"].str.lower()
metacriticData["Album"] = metacriticData["Album"].str.lower()

wikiData["Artist"] = wikiData["Artist"].str.lower()
metacriticData["Artist"] = metacriticData["Artist"].str.lower()

# Blocking via overlap

In [7]:
ob = em.OverlapBlocker()

#must match 3 criteria
#1. overlap at least 1 word of artist
oc = ob.block_tables(metacriticData, wikiData,'Artist','Artist',word_level=True,overlap_size=1,
                   l_output_attrs=["Album","Artist","Release Date"],
                   r_output_attrs=["Album","Artist","Release Date"],
                    show_progress=False)

print(len(oc))
# file_name = 'overlap_results.csv'
# C2.to_csv(file_name, sep=',')
# oc.head(1000)

96621


In [9]:
#2. at least 1 word of title
ob2 = em.OverlapBlocker()
oc2 = ob2.block_candset(oc, 'Album', 'Album', word_level=True,overlap_size=1,show_progress=False)
print(len(oc2))
#oc2.head(50)

#3. 1 of the 3 (month day year) of release date
ob3 = em.OverlapBlocker()
oc3 = ob3.block_candset(oc2, 'Release Date', 'Release Date', word_level=True,overlap_size=1,show_progress =False)
print(len(oc3))
oc3.head()


6975
3329


,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date
713,713,a2567,b44,feed the fire,promise and the monster,Jan 22 2016,the resistance: rise of the runaways,crown the empire,Jul 22 2014
715,715,a2569,b44,above the prairie,the pines,Jan 22 2016,the resistance: rise of the runaways,crown the empire,Jul 22 2014
724,724,a534,b44,"the sounds of crenshaw, vol. 1",the pollyseeds,Jul 14 2017,the resistance: rise of the runaways,crown the empire,Jul 22 2014
736,736,a557,b44,the usual suspects,jah wobble's invaders of the heart,Jul 7 2017,the resistance: rise of the runaways,crown the empire,Jul 22 2014
737,737,a558,b44,the nothing,the last dinosaur,Jul 7 2017,the resistance: rise of the runaways,crown the empire,Jul 22 2014


### Union results with album title equivalence blocker

In [14]:
#block with album title
ab = em.AttrEquivalenceBlocker()
C1 = ab.block_tables(metacriticData, wikiData, 
                   l_block_attr='Album', r_block_attr='Album', 
                    l_output_attrs=['Album', 'Artist', 'Release Date'],
                    r_output_attrs=['Album', 'Artist', 'Release Date'],
                    allow_missing=True)

G = em.combine_blocker_outputs_via_union([C1, oc3])
print len(G)

file_name = 'overlap_results.csv'
oc3.to_csv(file_name, sep=',')

6632


In [15]:
#G.head(50)

In [13]:
#dbg = em.debug_blocker(G,metacriticData,wikiData,output_size=50)
#dbg

# Blocking via attribute equivalence blocker

- apply multiple blockers to produce a candidate set of tuple pairs.
- assume that two albums with different release dates do not refer to the same real world album.
- then assume that two albums with different album names do not refer to the same real world album. So we apply attribute equivalence blocking on Album.

In [12]:
# Block with attribute equivalence blocker object
ab1 = em.AttrEquivalenceBlocker()

# block using release date
C1 = ab1.block_tables(metacriticData, wikiData, 
                   l_block_attr='Release Date', r_block_attr='Release Date', 
                    l_output_attrs=['Album', 'Artist', 'Release Date'],
                    r_output_attrs=['Album', 'Artist', 'Release Date'],
                    l_output_prefix='l_', r_output_prefix='r_', allow_missing=True)

In [10]:
# Instantiate attribute equivalence blocker object
ab2 = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C2 = ab2.block_candset(C1, 'Album', 'Album', show_progress = False)

In [13]:
# Display the candidate set of tuple pairs
len(C1)

51115

In [12]:
file_name = 'results.csv'
C2.to_csv(file_name, sep=',')

In [14]:
C2.head(10)

,_id,l_ID,r_ID,l_Album,l_Artist,l_Release Date,r_Album,r_Artist,r_Release Date
3,3,a16,b3181,Saturation III,BROCKHAMPTON,Dec 15 2017,Saturation III,Brockhampton,Dec 15 2017
19,19,a18,b3183,Revival,Eminem,Dec 15 2017,Revival,Eminem,Dec 15 2017
41,41,a21,b3184,The Beautiful & Damned,G-Eazy,Dec 15 2017,The Beautiful & Damned,G-Eazy,Dec 15 2017
51,51,a23,b3180,Asking Alexandria,Asking Alexandria,Dec 15 2017,Asking Alexandria,Asking Alexandria,Dec 15 2017
142,142,a34,b3176,What Makes You Country,Luke Bryan,Dec 8 2017,What Makes You Country,Luke Bryan,Dec 8 2017
175,175,a38,b3169,War & Leisure,Miguel,Dec 1 2017,War & Leisure,Miguel,Dec 1 2017
217,217,a43,b3171,Songs of Experience,U2,Dec 1 2017,Songs of Experience,U2,Dec 1 2017
336,336,a58,b3160,Who Built the Moon?,Noel Gallagher's High Flying Birds,Nov 24 2017,Who Built the Moon?,Noel Gallagher's High Flying Birds,Nov 24 2017
339,339,a59,b3154,Utopia,Björk,Nov 24 2017,Utopia,Björk,Nov 24 2017
376,376,a63,b3150,The Architect,Paloma Faith,Nov 17 2017,The Architect,Paloma Faith,Nov 17 2017


In [26]:
oc3.loc[oc3['ltable_Album'] == "Who Built the Moon?"]

,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date
96147,96147,a58,b3160,Who Built the Moon?,Noel Gallagher's High Flying Birds,Nov 24 2017,Who Built the Moon?,Noel Gallagher's High Flying Birds,Nov 24 2017
